In [ ]:
import random
from pathlib import Path
from urllib.parse import unquote

import bs4
import requests
from tqdm.notebook import tqdm

In [ ]:
import jsonpickle

ic = Path("ItemCache.json")
if not ic.exists():
    print("Performing Index")
    start_url = "https://runescape.wiki/w/Category:Item_inventory_images"
    scan_list = [start_url]
    item_pages = []
    broken = []

    while len(scan_list) > 0:
        item = scan_list.pop(0)
        print(item, len(item_pages))
        res = requests.get(
            item,
            headers={
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
            },
        )
        if res.status_code == 200:
            soup = bs4.BeautifulSoup(res.text)
            for link in soup.find_all("a"):
                if link.text == "next page":
                    scan_list.append("https://runescape.wiki" + link["href"])
                    break
            for item in soup.find("ul", {"class": "gallery"}).find_all("a"):
                img = item.find("img")
                if img is not None:
                    item = (item["href"], img.attrs["src"])
                    if item not in item_pages:
                        item_pages.append(item)
                else:
                    broken.append(item)
    ic.write_text(jsonpickle.encode(item_pages))
else:
    print("Index Loaded")
    item_pages = jsonpickle.decode(ic.read_text())
print(len(item_pages), "items")

In [ ]:
file_storage = Path("wiki_items/")
file_storage.mkdir(exist_ok=True)

In [ ]:
broken = []

In [ ]:
with tqdm(total=len(item_pages)) as progress:
    for page, file in item_pages:
        url = "https://runescape.wiki" + page
        try:
            name = unquote(url.split(":")[2].split(".")[0]).strip('"')
        except Exception as e:
            broken.append((page, file, e))
            continue
        if type(file) != str:
            file = file["href"]
        p = Path(file_storage, f"{name}.png")
        if not p.exists():
            progress.set_postfix_str(name)
            progress.update()
            res = requests.get(
                "https://runescape.wiki" + file,
                timeout=5,
                headers={
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
                },
            )
            p.write_bytes(res.content)
            continue
        progress.update()
print("done")

In [ ]:
len(broken)